In [ ]:
import sys
import jieba as jb
import re
from pyspark.sql import SparkSession

def createNameMap(filepath):
    names = []
    with open(filepath, 'r', encoding='utf-8') as fin:
        line = fin.readline()
        while line:
            line = line.strip('\n')
            names.append(line)
            line = fin.readline()
    name_dict = {}
    for name in names:
        name_dict[name] = 1
    return name_dict

def GetFileName(file_dir, suffix):
    fileNames =[]
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == suffix:
                fileNames.append(os.path.join(root, file))
    for i in range(0, len(fileNames)):
        fileNames[i] = fileNames[i].replace('\\', '/')
    return fileNames

def fetchName(text, name_dict):
    name_list = []
    for word in text:
        if word in name_dict:
            name_list.append(word)
    return name_list
        
def drive(taskName):
    spark = SparkSession\
        .builder\
        .appName(taskName)\
        .getOrCreate()
    
    filename_list = GetFileName('./../dataset/novels', '.txt')
    name_dict = createNameMap('./../dataset/people_name_list.txt')
    jb.load_userdict('./../dataset/people_name_list.txt')
    #inPath = './../dataset/novels/test.txt'
    for inPath in filename_list:
        print('Processing: ' + inPath)
        mapreduce(inPath, name_dict, spark)
    
    print('Please type any key to stop task.')
    input('')
    spark.stop()
    print('Done.')
    
def mapreduce(inPath, name_dict, spark):
    lines  = spark.read.text(inPath) \
            .rdd.map(lambda x:x[0]) \
            .map(lambda s: jb.lcut(s)) #此处不知为何会出现人名分词错误的情况 如'徐铮本'
    #print(lines.collect())
    names = lines.map(lambda x:fetchName(x, name_dict))
    output = names.collect()
    outPath = './../dataset/name/' + re.match(r'./../dataset/novels/(.*).txt', inPath).group(1) + '.name'
    fout = open(outPath, 'w', encoding='utf-8')
    for nameList in output:
        if(len(nameList) >= 1):
            fout.write('|'.join(nameList) + '\n')
    fout.close()
drive('task1')